In [63]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [64]:
train_df = pd.read_csv('../preprocessing_data/preprocessed_train.csv').drop('Id', axis= 1)
test_df = pd.read_csv('../preprocessing_data/preprocessed_test.csv').drop('Id', axis= 1)
X = np.array(train_df.drop(['SalePrice'], axis = 1).values)
y = np.log1p(np.array(train_df['SalePrice'].values))
X_test = np.array(test_df.values)

In [65]:
from bayes_opt import BayesianOptimization
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import get_scorer, mean_squared_error
import time

In [66]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [67]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state=42)

In [68]:
def rid_cl_bo(alpha, max_iter, tol):
    params_rid = {}
    # sol = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    params_rid['alpha'] = round(alpha)
    params_rid['max_iter'] = round(max_iter)
    params_rid['tol'] = tol
    
    # params_rid['solver'] = sol[round(solver)]
    
    rid = Ridge(**params_rid)
    rid.fit(X_train, y_train)

    score = -rmse(y_valid, rid.predict(X_valid))
    return score

# Run Bayesian Optimization
start = time.time()
params_rid ={
    'alpha':(1, 25),
    'max_iter':(1000, 3000),
    'tol':(0.0001, 0.001),
    # 'solver': (0, 6),
}
rid_bo = BayesianOptimization(rid_cl_bo, params_rid, random_state=111)
rid_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   |   alpha   | max_iter  |    tol    |
-------------------------------------------------------------
| 1         | -0.1203   | 15.69     | 1.338e+03 | 0.0004925 |
| 2         | -0.1204   | 19.46     | 1.591e+03 | 0.0002342 |
| 3         | -0.121    | 1.539     | 1.84e+03  | 0.0003148 |
| 4         | -0.1202   | 9.104     | 2.981e+03 | 0.000314  |
| 5         | -0.1207   | 2.949     | 2.339e+03 | 0.0006591 |
| 6         | -0.1202   | 7.582     | 1.932e+03 | 0.0002065 |
| 7         | -0.1207   | 2.775     | 2.802e+03 | 0.0008146 |
| 8         | -0.1205   | 21.17     | 2.63e+03  | 0.0009919 |
| 9         | -0.1203   | 14.85     | 2.628e+03 | 0.0004792 |
| 10        | -0.121    | 1.659     | 1.908e+03 | 0.0001948 |
| 11        | -0.1205   | 20.61     | 2.395e+03 | 0.0006088 |
| 12        | -0.1202   | 7.581     | 2.997e+03 | 0.0002242 |
| 13        | -0.1203   | 15.77     | 1.97e+03  | 0.0004645 |
| 14        | -0.1204   | 18.47     | 1.646e+03 | 0.0004605 |
| 15    

In [71]:
rid = Ridge(alpha = 10, max_iter = 2895, tol= 0.0007341)
rid.fit(X_train, y_train)
rmse(y_valid, rid.predict(X_valid))

0.12019752671912572

In [72]:
ans = pd.read_csv('../submission/cheat.csv').drop('Id', axis = 1)
ans = np.array(ans.values).reshape(1, -1)[0]
y_pred = rid.predict(X_test)
print(f"Error: {rmse(np.log1p(ans), y_pred)}")

Error: 0.12682236986758766
